In [1]:
import sys
sys.path.append('C:\\Users\\Hasegawa\\Desktop\\AIT Note\\Thesis\\thesis-code\\Soft-Actor-Critic')

In [2]:
import os

import numpy as np
import torchvision.transforms as T
import matplotlib.pyplot as plt

from itertools import cycle
from PIL import Image, ImageDraw, ImageFont

from common.network import ConvVAE

In [3]:
image_size = (270, 480)
H, W = image_size

In [4]:
weight_base_path = '..\\vae_weights'
models_dict = {
    # 'b1': (1, 'Carla-v0_town7_outskirts_b1//bvae_town7_epoch(50)-loss(+2.503E+05).pkl'),
    'b2': (2, 'Carla-v0_town7_outskirts_b2//bvae_town7_epoch(100)-loss(+2.451E+05).pkl'),
    'b3': (3, 'Carla-v0_town7_outskirts_b3//bvae_town7_epoch(100)-loss(+2.453E+05).pkl'),
    'b4': (4, 'Carla-v0_town7_outskirts_b4//bvae_town7_epoch(100)-loss(+2.455E+05).pkl')
}

In [5]:
base_path = 'C:\\Users\\Hasegawa\\Desktop\\AIT Note\\Thesis\\episode_observation'
image_paths = list(map(lambda name: os.path.join(base_path, name), os.listdir(base_path)))

In [6]:
to_tensor = T.ToTensor()

def out_tensor_to_np_image(out_tensor):
    ''' Convert output tensor to numpy image '''
    out_img_np = out_tensor[0].squeeze(dim=0).detach().numpy()
    out_img_np = (out_img_np.transpose(1, 2, 0) * 255).astype('uint8')
    return out_img_np

In [7]:
reconstruct_images = []
for key, (beta, weight_name) in models_dict.items():
    latent_size = 512
#     if key == 'b3':
#         latent_size = 1024
    model = ConvVAE(image_size, latent_size=latent_size, beta=beta)
    model.load_model(os.path.join(weight_base_path, weight_name))
    
    images = []
    for image_path in image_paths:
        pil_img = Image.open(image_path)
        np_img = np.array(pil_img)
        in_tensor = to_tensor(pil_img)
        
        out_tensor = model(in_tensor.unsqueeze(dim=0))
        out_np_img = out_tensor_to_np_image(out_tensor)
        
        combined_np_img = np.hstack([np_img, out_np_img])
        images.append(combined_np_img)
        
        pil_img.close()
        
    reconstruct_images.append(images)

In [8]:
for num, images in enumerate(zip(*reconstruct_images), 1):
    combined_np_img = np.vstack(images)
    combined_pil_img = Image.fromarray(combined_np_img)

    draw = ImageDraw.Draw(combined_pil_img)
    
    pos_y = 0
    margin = 10
    
    for beta in range(2, 5):
        text = f'Beta {beta}'
        # textwidth, textheight = draw.textsize(text)

        pos_y = pos_y + margin
        draw.text((10, pos_y), text)
        
        pos_y += H

    combined_pil_img.save(f'.\\visualize observation reconstruction\\result_{num}.jpeg')